In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras import layers
from keras import models
from keras.models import Sequential
from keras.layers import Concatenate, Activation, BatchNormalization, Flatten
from keras.layers import Input, Conv2D, MaxPooling2D, Input, AveragePooling2D, Dense
from keras.models import Model
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras import optimizers
import time

Using TensorFlow backend.


In [2]:
def add_inception(input):
    B1 = Conv2D(128,(1,1), padding='same')(L1)
    B1 = Activation('relu')(B1)
    B2 = Conv2D(16,(1,1), padding='same')(L1)
    B2 = Activation('relu')(B2)
    B2 = Conv2D(32,(3,3), padding='same')(L1)
    B2 = Activation('relu')(B2)
    B3 = Conv2D(96,(1,1), padding='same')(L1)
    B3 = Activation('relu')(B2)
    B3 = Conv2D(128,(3,3), padding='same')(L1)
    B3 = Activation('relu')(B2)
    #B4 = Conv2D(64,(1,1), padding='same')(L1)
    #B4 = Activation('relu')(B4)
    #L2 = layers.concatenate([B1, B2, B3, B4], axis=3)
    L2 = layers.concatenate([B1, B2, B3], axis=3)
    return L2

In [3]:
main_input = Input(shape=(32,32,3))
L1 = Conv2D(32, (3, 3), padding='same')(main_input)
L1 = BatchNormalization()(L1)
L1 = Activation('relu')(L1)
L1 = Conv2D(32, (1, 1), padding='same')(L1)
L1 = Activation('relu')(L1)
L1 = Conv2D(32, (3, 3), padding='same')(L1)
L1 = Activation('relu')(L1)
L1 = BatchNormalization()(L1)
L1 = MaxPooling2D(pool_size=(3,3))(L1)

#B1 = MaxPooling2D(pool_size=(2,2))(L1)
# Inception Module
L1 = add_inception(L1)
L1 = add_inception(L1)
L1 = add_inception(L1)
L1 = add_inception(L1)
L1 = add_inception(L1)

L2 = AveragePooling2D(pool_size=(2,2))(L1)
L2 = Flatten()(L2)
L2 = Dense(128)(L2)
L2 = Dense(10,activation='softmax')(L2)

In [4]:
model = Model(main_input, L2)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [5]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255 
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test) 
sgd = optimizers.SGD(lr=0.01, decay=1e-4, momentum=0.9, nesterov=True) 
start_time = time.time()
model.compile(optimizer=sgd, 
              loss='categorical_crossentropy', 
              metrics=['accuracy']) 
end_time = time.time()
history = model.fit(x_train, y_train, epochs=5, shuffle='batch', batch_size=64) 

170500096/170498071 [==============================] - 20s 0us/step
Epoch 1/5
50000/50000 [==============================] - 31s 626us/step - loss: 1.2617 - acc: 0.5487
Epoch 2/5
50000/50000 [==============================] - 28s 555us/step - loss: 0.8485 - acc: 0.7022
Epoch 3/5
50000/50000 [==============================] - 28s 555us/step - loss: 0.6931 - acc: 0.7566
Epoch 4/5
50000/50000 [==============================] - 28s 553us/step - loss: 0.5904 - acc: 0.7928
Epoch 5/5
50000/50000 [==============================] - 28s 568us/step - loss: 0.5076 - acc: 0.8217


In [6]:
test_loss, test_acc = model.evaluate(x_test, y_test) 
print(test_loss, test_acc)

10000/10000 [==============================] - 2s 236us/step
0.7093710705280304 0.7624


In [12]:
training_time = end_time - start_time